<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting/blob/main/myProspects_Valuation_Gap_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2024-03-25 16:49:39


In [2]:
!pip install yfinance

In [3]:
import pandas as pd
import numpy as np

mon = 'Dec'

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')
#mypps = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myProspectsScrips.csv')

mypfs = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting/main/data/myPortfolioStocks.csv')
mypfs = mypfs[mypfs['InPortfolio'].isin(['SV','DM'])]['Symbol']
mypfs = mypps[mypps['Symbol'].isin(mypfs)]['Stock']

#mypf = mypf[(mypf['Criteria'] == 'V40') | (mypf['Criteria'] == 'V40N')]
mypps = mypps[mypps['Criteria'].isin(['V40','V40N','VR','AR', 'V200'])]

stock_n100 = mypps['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','PGHH','SIEMENS', 'GILLETTE', 'NESTLEIND', 'SANOFI', 'AMARAJABAT', 'SCHAEFFLER ']

stock_n100 = mypps[~mypps['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100[:]
#stock_ids = ['HDFCBANK', 'COLPAL']
print('Stocks to be scanned:',len(stock_ids))

Stocks to be scanned: 134


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids:


    #print(stock_name)

    link_blueprint = "https://www.screener.in/company/"

    stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL' , 'IEX', 'ROSSARI', 'VALIANTORG', 'VINATIORGA', 'TATAELXSI',
                  'AMARAJABAT', 'EQUITAS', 'AAVAS', 'CAMS', 'DREAMFOLKS', 'LICI']

    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list
    #print(actual_link)
    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list)

    if n < 100:
        actual_link = link_blueprint + stock_name.upper()
        test_list = get_list(actual_link)
        #print(len(test_list))

    #print(test_list)
    if 'TTM' not in test_list:
        end_year = 'Mar 2023'
    else:
        end_year = 'TTM'

        df_yoy = pd.DataFrame()
        #print(test_list)
        s = test_list.index(end_year) +1 - test_list.index('Mar 2018')

        df_yoy['Year'] = test_list[test_list.index('Mar 2018'):test_list.index(end_year)+1]
        df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
        df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

        df_yoy.set_index('Year', inplace=True)
        df_yoy = df_yoy.T
        df_yoy['Stock'] = stock_name.upper() + '.NS'
        #df_yoy
        df_latest_analysis = pd.concat([df_latest_analysis, df_yoy])

df_latest_analysis.index.names = ['Type']
df_latest_analysis.reset_index(inplace=True)
df_latest_analysis.rename(columns={'TTM': mon+' 2023'}, inplace=True)
#df_latest_analysis.drop('TTM', axis=1, inplace=True)
cols = [ 'Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', mon+' 2023']
for col in cols:
    df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

In [5]:
df_latest_analysis

Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Dec 2023,Stock
0,Sales,2556.0,2809.0,2765.0,2420.0,3078.0,3733.0,3881.0,3MINDIA.NS
1,Profit,480.0,497.0,320.0,202.0,360.0,561.0,671.0,3MINDIA.NS
2,Sales,20.0,61.0,108.0,195.0,298.0,338.0,372.0,5PAISA.NS
3,Profit,-33.0,-25.0,-10.0,20.0,18.0,58.0,84.0,5PAISA.NS
4,Sales,3806.0,4168.0,4186.0,4506.0,6086.0,6619.0,6255.0,AARTIIND.NS
...,...,...,...,...,...,...,...,...,...
257,Profit,541.0,626.0,653.0,451.0,304.0,247.0,206.0,WHIRLPOOL.NS
258,Sales,54487.0,59019.0,61138.0,61935.0,79312.0,90488.0,90742.0,WIPRO.NS
259,Profit,10242.0,11542.0,12252.0,13903.0,15141.0,14766.0,14877.0,WIPRO.NS
260,Sales,1710.0,1979.0,2493.0,2245.0,3378.0,4921.0,4735.0,WSTCSTPAPR.NS


In [6]:
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()

stock_df_final = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  short_window = 20
  mid_window = 50
  long_window = 200
  moving_avg = 'SMA'

  # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
  stock_df = yfin.Ticker(stock_symbol).history(period='7y', interval='1mo')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
  stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
  stock_df.dropna(axis = 0, inplace = True) # remove any null rows

  stock_df['Stock'] = stock_symbol

  stock_df['Close'] = round(stock_df['Close'],2)
  stock_df['Mon'] = stock_df.index.month_name().str.slice(stop=3)
  stock_df['Year'] = stock_df.index.strftime('%Y')
  stock_df['Year'] = stock_df['Mon'] + ' ' + stock_df['Year']
  cols = [ 'Year', 'Close']
  stock_df = stock_df[cols]
  stock_df = stock_df[(stock_df['Year'].str.contains("Mar")) | (stock_df['Year'].str.contains(mon+' 2023'))]
  stock_df = stock_df.set_index('Year')
  stock_df = stock_df.T
  stock_df['Stock'] = stock_symbol
  stock_df.index.names = ['Type']
  stock_df = stock_df.reset_index()
  stock_df_final = pd.concat([stock_df_final, stock_df])

stock_df_final

Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Dec 2023,Mar 2024,Stock
0,Close,18667.90,23329.87,18105.45,29268.68,18941.44,22882.53,37309.15,30798.00,3MINDIA.NS
0,Close,203.15,144.82,113.15,249.70,344.70,283.00,566.25,490.00,5PAISA.NS
0,Close,272.25,375.83,378.50,651.85,950.19,516.46,649.60,655.00,AARTIIND.NS
0,Close,NaN,1155.80,1188.85,2420.85,2569.45,1610.75,1532.25,1351.20,AAVAS.NS
0,Close,5280.31,7078.48,14997.24,14666.08,17483.04,21929.65,22851.25,27049.00,ABBOTINDIA.NS
...,...,...,...,...,...,...,...,...,...,...
0,Close,308.00,470.79,236.75,349.14,735.10,570.60,595.46,463.70,VIPIND.NS
0,Close,NaN,NaN,321.81,458.00,477.58,680.83,816.85,758.60,WESTLIFE.NS
0,Close,1485.14,1493.49,1798.81,2207.28,1562.80,1310.90,1366.85,1235.40,WHIRLPOOL.NS
0,Close,205.94,249.56,193.43,408.18,584.32,364.47,470.30,488.15,WIPRO.NS


In [7]:
stock_df_final.drop(['Dec 2023',], axis=1, inplace=True)
stock_df_final.rename(columns={'Mar 2024':'Dec 2023'}, inplace=True)

In [8]:
df_gap_analysis = df_latest_analysis.append(stock_df_final)
df_gap_analysis['Growth%'] = (df_gap_analysis[mon+' 2023'] - df_gap_analysis['Mar 2018'])*100/df_gap_analysis['Mar 2018']
df_gap_analysis['Growth%'] = round(df_gap_analysis['Growth%'],2)
df_gap_analysis

Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Dec 2023,Stock,Growth%
0,Sales,2556.00,2809.00,2765.00,2420.00,3078.00,3733.00,3881.00,3MINDIA.NS,51.84
1,Profit,480.00,497.00,320.00,202.00,360.00,561.00,671.00,3MINDIA.NS,39.79
2,Sales,20.00,61.00,108.00,195.00,298.00,338.00,372.00,5PAISA.NS,1760.00
3,Profit,-33.00,-25.00,-10.00,20.00,18.00,58.00,84.00,5PAISA.NS,-354.55
4,Sales,3806.00,4168.00,4186.00,4506.00,6086.00,6619.00,6255.00,AARTIIND.NS,64.35
...,...,...,...,...,...,...,...,...,...,...
0,Close,308.00,470.79,236.75,349.14,735.10,570.60,463.70,VIPIND.NS,50.55
0,Close,NaN,NaN,321.81,458.00,477.58,680.83,758.60,WESTLIFE.NS,NaN
0,Close,1485.14,1493.49,1798.81,2207.28,1562.80,1310.90,1235.40,WHIRLPOOL.NS,-16.82
0,Close,205.94,249.56,193.43,408.18,584.32,364.47,488.15,WIPRO.NS,137.04


In [11]:
stock_df_gap = pd.DataFrame()

for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  #print(stock_symbol)
  tmp_df = df_gap_analysis[df_gap_analysis['Stock'] == stock_symbol]
  #print(tmp_df)
  try:
    gap_perc = round(tmp_df[tmp_df['Type'] == 'Profit']['Growth%'].values[0]  - tmp_df[tmp_df['Type'] == 'Close']['Growth%'].values[0],2)
  except:
    pass
  #print(gap_perc)
  tmp_df = pd.DataFrame()
  tmp_df['Stock'] = [stock_symbol]
  tmp_df['Gap%'] = [gap_perc]
  stock_df_gap = pd.concat([stock_df_gap, tmp_df])

cols = ['Stock', 'Criteria']
tmp_df = mypps[cols]
stock_df_gap = pd.merge(stock_df_gap, tmp_df,  on='Stock')
stock_df_gap.sort_values(['Gap%'], ascending=[False]).head(30)

,Stock,Gap%,Criteria
13,AXISBANK.NS,3691.26,V40
116,TANLA.NS,2317.10,AR
117,TATACOMM.NS,2285.63,AR
71,J&KBANK.NS,1586.35,VR
133,WSTCSTPAPR.NS,319.46,VR
59,ICICIBANK.NS,135.26,V40
64,INDUSINDBK.NS,123.57,VR
22,BAYERCROP.NS,121.02,V40N
48,GICRE.NS,113.16,VR
77,KANSAINER.NS,111.69,V40N


In [12]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    #start = datetime.datetime(*map(int, start_date.split('-')))
    #end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol).history(period='6y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    #stock_df.columns = ['Close']

    #stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Stock'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Max'] = round(max(stock_df['Close']),2)
    #stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
    #stock_df['Min'] = round(min(stock_df['Close']),2)
    #stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    #stock_df['Close'] = round(stock_df['Close'],0)
    #stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
    #stock_df['Sector'] = stock_symbol
    stock_df = stock_df.tail(1)
    cols = ['Stock', 'ATH%']
    return stock_df[cols]

In [13]:
stock_df_max = pd.DataFrame()
for stock_name in stock_ids:

  stock_symbol = stock_name + '.NS'
  tmp_df = stock_prec_dev(stock_symbol)
  stock_df_max = pd.concat([stock_df_max, tmp_df])
stock_df_max

,Stock,ATH%
Date,,
2024-03-22 00:00:00+05:30,3MINDIA.NS,21.42
2024-03-22 00:00:00+05:30,5PAISA.NS,50.88
2024-03-22 00:00:00+05:30,AARTIIND.NS,74.93
2024-03-22 00:00:00+05:30,AAVAS.NS,135.66
2024-03-22 00:00:00+05:30,ABBOTINDIA.NS,9.13
...,...,...
2024-03-22 00:00:00+05:30,VIPIND.NS,60.03
2024-03-22 00:00:00+05:30,WESTLIFE.NS,29.94
2024-03-22 00:00:00+05:30,WHIRLPOOL.NS,114.81


In [17]:
stock_df_final = pd.merge(stock_df_gap, stock_df_max,  on='Stock')
stock_df_final['InPF'] = 0
stock_df_final.loc[stock_df_final['Stock'].isin(mypfs), 'InPF'] = 1
stock_df_final[(stock_df_final['Gap%'] >=50) & (stock_df_final['Gap%'] <= 300)].sort_values(by = 'ATH%', ascending=False)

,Stock,Gap%,Criteria,ATH%,InPF
84,LUXIND.NS,54.75,V40N,306.18,1
36,CUB.NS,59.44,AR,76.79,0
77,KANSAINER.NS,111.69,V40N,63.61,1
111,SIS.NS,102.69,V40N,55.06,1
65,INFIBEAM.NS,55.94,AR,40.08,0
114,SUNTV.NS,66.24,V40N,39.48,1
43,EMAMILTD.NS,106.61,V200,36.26,1
48,GICRE.NS,113.16,VR,31.59,0
64,INDUSINDBK.NS,123.57,VR,29.95,1
30,CAPLIPOINT.NS,58.87,V40N,25.93,0


In [15]:
df_gap_analysis[df_gap_analysis['Stock']  == 'TATACOMM.NS']

Year,Type,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Dec 2023,Stock,Growth%
228,Sales,16772.00,16525.00,17068.00,17100.00,16725.00,17838.00,19846.0,TATACOMM.NS,18.33
229,Profit,46.00,343.00,140.00,1609.00,2000.00,2063.00,1301.0,TATACOMM.NS,2728.26
0,Close,355.86,356.07,221.64,1014.82,1185.89,1228.74,1931.0,TATACOMM.NS,442.63
